In [10]:
from torchvision.models import ResNet50_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import datasets
import torch
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np

ImportError: cannot import name 'transform' from 'torchvision' (/Users/pauladler/MPDL_Project_1/.venv/lib/python3.9/site-packages/torchvision/__init__.py)

In [2]:
# Set device
device = "cpu"
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda_is_available():
    device = "cuda"

device = torch.device(device)

#Set Parameters for creating the Dataset
num_workers = 0
crop_size = 256

# Set hyperparameters for training
epochs = 100
batch_size = 32
learning_rate = 0.001
weight_decay = 0.0001
patience = 5

In [3]:
def to_3_channels(image):
    if image.shape[0] == 3:
        return image
    elif image.shape[0] == 1:
        return image.repeat(3, 1, 1)
    else:
        # Select the first 3 channels if the input has more than 3 channels
        return image[:3, :, :]

def collate_fn(examples):
    images, labels = [], []

    image_transform = T.Compose([
        # apply random square crop with max possible size of the current image, then resize to crop_sizexcrop_size
        T.RandomCrop(crop_size, pad_if_needed=True),
        T.Resize((crop_size,crop_size)),
        T.ToTensor(),
        T.Lambda(to_3_channels),
        ])

    # Iterate through the examples, apply the image transformation, and append the results
    for example in examples:
        image = image_transform(example['image'])
        label = example['label']
        images.append(image)
        labels.append(label)

        pixel_values = torch.stack(images)
    labels = torch.tensor(labels)

    return {"pixel_values": pixel_values, "label": labels}


class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf
        self.min_model = None

    def early_stop(self, validation_loss, model):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.min_model = model
            self.counter = 0
        elif validation_loss >= (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:

train_dataset = datasets.load_dataset("Hanneseh/MPDL_Project_1", split="train")
val_dataset = datasets.load_dataset("Hanneseh/MPDL_Project_1", split="validation")

In [ ]:
train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, num_workers=num_workers, shuffle=True)

In [4]:
model = torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
model.fc = torch.nn.Linear(2048, 2)
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
stopper = EarlyStopper(patience=patience, min_delta=0)

In [ ]:
train_accu, train_losses = [], []

def train(data):
    correct, running_loss, total = 0, 0, 0

    model.train()
    for element in tqdm(data):
        # Move input and label tensors to the device
        inputs = element["pixel_values"].to(device)
        labels = element["label"].to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss=running_loss/len(data)
    accu=100.*correct/total

    train_accu.append(accu)
    train_losses.append(train_loss)
    print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))
    return train_loss


In [ ]:
stop = False
eval_losses, eval_accu=[], []

def val(data):
    correct, running_loss, total = 0, 0, 0

    model.eval()
    with torch.no_grad():
      for element in tqdm(data):
          inputs = element["pixel_values"].to(device)
          labels = element["label"].to(device)
          
        
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          running_loss += loss.item()

          _, predicted = outputs.max(1)
          total += labels.size(0)
          correct += predicted.eq(labels).sum().item()

    val_loss=running_loss/len(data)
    accu=100.*correct/total

    eval_accu.append(accu)
    eval_losses.append(val_loss)
    print('Val Loss: %.3f | Accuracy: %.3f'%(val_loss,accu))
    return val_loss



In [ ]:
for epoch in range(1,epochs+1):
  print('\nEpoch : %d'%epoch)
  train_loss = train(train_dataloader)
  val_loss = val(val_dataloader)
  if stopper.early_stop(val_loss, model):
    model = stopper.min_model
    break

In [ ]:
test_dataset = datasets.load_dataset("Hanneseh/MPDL_Project_1", split="test")
test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=batch_size, num_workers=num_workers, shuffle=True)
print("")
model.eval()
with torch.no_grad():
    correct, total = 0, 0
    for element in tqdm(test_dataloader):
        inputs = element["pixel_values"].to(device)
        labels = element["label"].to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [ ]:
transform = T.Compose([
    T.RandomCrop(crop_size, pad_if_needed=True),
    T.Resize((crop_size, crop_size)),
    T.ToTensor()
])

PATH = './classifier.pth'

img = Image.open('pictures/test.jpg')
img = transform(img)[None, :, :, :]

model = model.to("cpu")
model.eval()
with torch.no_grad():
    traced = torch.jit.trace(model, img)
torch.jit.save(traced, PATH)

In [ ]:
plt.plot(train_accu,'-o')
plt.plot(eval_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')

plt.show()

In [ ]:
plt.plot(train_losses,'-o')
plt.plot(eval_losses,'-o')
plt.xlabel('epoch')
plt.ylabel('losses')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Losses')

plt.show()

In [21]:
PATH = './classifier.pth'

transform = T.Compose([
    T.RandomCrop(crop_size, pad_if_needed=True),
    T.Resize((crop_size, crop_size)),
    T.ToTensor()
])

img = Image.open('pictures/test.jpg')
img = transform(img)[None, :, :, :]

reloaded = torch.jit.load(PATH)
reloaded.eval()
with torch.no_grad():
    pred = reloaded(img)
    prob = torch.nn.Softmax(dim=1)
    print(pred)
    print(prob(pred))

torch.Size([1, 2])
tensor([[0.1198, 0.8802]])
